In [1]:
%pip install "zenml[server]"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2

import IPython

# IPython.Application.instance().kernel.do_shutdown(restart=True)

Defaulting to user installation because normal site-packages is not writeable


Note: you may need to restart the kernel to use updated packages.
/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.
⠸ Installing integrations.....
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
NGROK_TOKEN = "2FrdinggJCMYB098glpY73WoD4f_6hFdEVRnTJcbgjNxJiyRY"

In [3]:
# from zenml.environment import Environment

# if Environment.in_google_colab():
    
#     # Install and authenticate ngrok
#     !pip install pyngrok
#     !ngrok authtoken {NGROK_TOKEN}

In [4]:
!pip install pyngrok
!ngrok authtoken {NGROK_TOKEN}

Defaulting to user installation because normal site-packages is not writeable
Authtoken saved to configuration file: /home/unicconaiadmin/.config/ngrok/ngrok.yml                 


In [6]:
## ZEN SETUP

!rm -rf .zen
!zenml init

/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.
Initializing the ZenML global configuration version to 0.56.2
Creating database tables
⠋ Initializing ZenML repository at /mnt/SDD1/zenml.
⠙ Initializing ZenML repository at /mnt/SDD1/zenml.
⠹ Initializing ZenML repository at /mnt/SDD1/zenml.
⠸ Initializing ZenML repository at /mnt/SDD1/zenml.
Creating default workspace 'default' ...
Creating default stack in workspace default...
Creating default user 'default' ...
⠼ Initializing ZenML repository at /mnt/SDD1/zenml.
⠴ Initializing ZenML repository at /mnt/SDD1/zenml.
Setting the global active workspace to 'default'.
Setting the global active stack to default.
Setting the repo active workspace to 'default'.
Setting the repo active stack to default.
⠦ Initializi

In [9]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split


def train_test() -> None:
    """ Training and testing a simple SVC model"""
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    X_train, X_test, y_train, y_test = train_test_split(data, digits.target, shuffle=False, test_size=0.2)
    model = SVC(gamma=0.001)
    model.fit(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    print(f"Test accuracy: {test_acc}")
    
    
train_test()

Test accuracy: 0.9583333333333334


In [10]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple


@step
def importer() -> Tuple[Annotated[
    np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "y_test"]
]:
    """Load the digits dataset as a numpy array"""
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    X_train, X_test, y_train, y_test = train_test_split(
        data, digits.target, test_size=0.2, shuffle=False
        )
    return X_train, X_test, y_train, y_test


@step
def svc_trainer(
    X_train: np.ndarray,
    y_train: np.ndarray
) -> ClassifierMixin:
    """ Training a SVC model"""
    model = SVC(gamma=0.001)
    model.fit(X_train, y_train)
    return model


@step
def evaluator(
    X_test: np.ndarray,
    y_test: np.ndarray,
    model: ClassifierMixin
) -> float:
    """This helps to evaluate the performance of the model"""
    test_acc = model.score(X_test, y_test)
    print(f"Test Accuracy: {test_acc}")
    return test_acc

In [11]:
from zenml import pipeline


@pipeline
def digits_pipeline():
    """This links all steps in the pipeline together"""
    X_train, X_test, y_train, y_test = importer()
    model = svc_trainer(X_train=X_train, y_train=y_train)
    evaluator(X_test=X_test, y_test=y_test, model=model)

In [12]:
digits_svc_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.
Registered new version: (version 1).
Executing a new run.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.
Step importer has started.
/home/unicconaiadmin/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
Step importer has finished in 0.960s.
Step svc_trainer has started.
Step svc_trainer has finished in 0.167s.
Step evaluator has started.
Test Accuracy: 0.9583333333333334
Step evaluator has finished in 0.871s.
Pipeline run has finished in 2.104s.
